# **과제3**

In [13]:
import pandas as pd
from IPython.display import display

# ✅ 최대 힙 클래스 정의
class Heap:
    def __init__(self, *args):
        if len(args) != 0:
            self.__A = args[0]
        else:
            self.__A = []

    def insert(self, x):
        self.__A.append(x)
        self.__percolateUp(len(self.__A) - 1)

    def __percolateUp(self, i: int):
        parent = (i - 1) // 2
        if i > 0 and self.__A[i] > self.__A[parent]:
            self.__A[i], self.__A[parent] = self.__A[parent], self.__A[i]
            self.__percolateUp(parent)

    def deleteMax(self):
        if not self.isEmpty():
            max = self.__A[0]
            self.__A[0] = self.__A.pop()
            self.__percolateDown(0)
            return max
        else:
            return None

    def __percolateDown(self, i: int):
        child = 2 * i + 1
        right = 2 * i + 2
        if child <= len(self.__A) - 1:
            if right <= len(self.__A) - 1 and self.__A[child] < self.__A[right]:
                child = right
            if self.__A[i] < self.__A[child]:
                self.__A[i], self.__A[child] = self.__A[child], self.__A[i]
                self.__percolateDown(child)

    def isEmpty(self):
        return len(self.__A) == 0

    def getHeap(self):
        return self.__A

# ✅ CSV 파일 업로드 코드 (Colab 환경 전용)
from google.colab import files
uploaded = files.upload()

# ✅ 파일 이름과 경로 지정
csv_path = "DS_Birthdaydata - 시트1.csv"
df = pd.read_csv(csv_path)

# ✅ 최대 힙 생성 후 (생일, 이름) 삽입
heap = Heap()
for _, row in df.iterrows():
    try:
        birth = int(row["생년월일8자리(예.20040101)"])
        name = row["이름"]
        heap.insert((birth, name))
    except:
        continue  # 생일이 NaN이거나 오류 있으면 건너뜀

# ✅ 최신 생일 기준으로 상위 10명 추출
top_10 = []
for _ in range(10):
    if not heap.isEmpty():
        birth, name = heap.deleteMax()
        top_10.append({"이름": name, "생년월일": birth})

# ✅ 결과 출력 (Colab 셀에 깔끔하게 보이도록)
top_10_df = pd.DataFrame(top_10)
print("🎉 생일이 가장 늦은 친구 10명 (최신 순):")
display(top_10_df)

Saving DS_Birthdaydata - 시트1.csv to DS_Birthdaydata - 시트1 (2).csv
🎉 생일이 가장 늦은 친구 10명 (최신 순):


,이름,생년월일
0,홍서연,20241282
1,신수민,20051230
2,이서영,20051225
3,강민주,20051214
4,김민경,20051202
5,이서영,20051112
6,배시은,20051102
7,김여원,20051031
8,이서진,20051028
9,서홍빈,20051024


##코드해설
CSV 업로드 방식->files.upload() 사용하여 Colab에서 직접 업로드 가능하도록 처리!!

1. heap.py 코드 사용	✅	힙 클래스 직접 구현되어 있음 (insert, deleteMax, percolateUp/Down)
2. 생일 데이터를 힙에 저장	✅	(birth, name) 형식으로 insert() 수행
3. 최신 생일이 위로 오도록 최대 힙 구현	✅	생일(숫자)을 기준으로 비교 → 큰 값이 위로 올라감
4. 상위 10명 추출	✅	deleteMax()를 최대 10번 반복
5. 결과를 Colab 셀에 출력	✅	display(top_10_df) 사용 → Colab 셀에 표 형태로 출력됨
6. 생일이 누락된 경우 예외 처리	✅	try-except로 NaN 혹은 오류 안전하게 skip
*추가

from IPython.display import display 사용 → Colab 환경 대응

files.upload() 포함 → 직접 파일 업로드까지 고려되어 있음

예외 발생 시 continue 처리하여 데이터 오류에 유연하게 대응함



#**과제4**

In [12]:
import pandas as pd
from IPython.display import display
from google.colab import files

# 📂 Colab에서 파일 업로드
uploaded = files.upload()

# -------------------------------
# BidirectNode 클래스
# -------------------------------
class BidirectNode:
    def __init__(self, x, prevNode: 'BidirectNode', nextNode: 'BidirectNode'):
        self.item = x
        self.prev = prevNode
        self.next = nextNode

# -------------------------------
# CircularDoublyLinkedList 클래스
# -------------------------------
class CircularDoublyLinkedList:
    def __init__(self):
        self.__head = BidirectNode("dummy", None, None)
        self.__head.prev = self.__head
        self.__head.next = self.__head
        self.__numItems = 0

    def append(self, newItem) -> None:
        prev = self.__head.prev
        newNode = BidirectNode(newItem, prev, self.__head)
        prev.next = newNode
        self.__head.prev = newNode
        self.__numItems += 1

    def getNode(self, i: int) -> BidirectNode:
        curr = self.__head
        for index in range(i + 1):
            curr = curr.next
        return curr

    def __iter__(self):
        return CircularDoublyLinkedListIterator(self)

# -------------------------------
# 반복자 클래스
# -------------------------------
class CircularDoublyLinkedListIterator:
    def __init__(self, alist):
        self.__head = alist.getNode(-1)
        self.iterPosition = self.__head.next

    def __next__(self):
        if self.iterPosition == self.__head:
            raise StopIteration
        else:
            item = self.iterPosition.item
            self.iterPosition = self.iterPosition.next
            return item

# -------------------------------
# 데이터 처리 및 출력
# -------------------------------

# ✅ 업로드한 파일 이름 그대로 사용
csv_path = "DS_Birthdaydata - 시트1.csv"
df = pd.read_csv(csv_path)

# 조원 목록
group_members = ["선예영", "황채원", "나현주", "윤서현", "이승주",
                 "조승연", "남수연", "김민선", "김하린", "이서진"]

# 리스트에 데이터 저장
birthday_list = CircularDoublyLinkedList()
for _, row in df.iterrows():
    name = row["이름"]
    student_id = str(row["학번"]).strip()
    birthday = row["생년월일8자리(예.20040101)"]
    birthday_list.append({"이름": name, "학번": student_id, "생년월일": birthday})

# 결과 저장용 리스트
valid_births = []
nan_births = []

# 학번이 44로 끝나는 이서진 먼저 추가
for entry in birthday_list:
    if entry["이름"] == "이서진" and entry["학번"].endswith("44"):
        birth = entry["생년월일"]
        birth_value = int(birth) if pd.notna(birth) else ""
        valid_births.append({"이름": entry["이름"], "생년월일": birth_value})

# 나머지 조원 처리
for name in group_members:
    if name == "이서진":
        continue
    match = df[df["이름"] == name]
    if not match.empty:
        row = match.iloc[0]
        birthday = row["생년월일8자리(예.20040101)"]
        birth_value = int(birthday) if pd.notna(birthday) else ""
        if birth_value != "":
            valid_births.append({"이름": name, "생년월일": birth_value})
        else:
            nan_births.append({"이름": name, "생년월일": ""})
    else:
        nan_births.append({"이름": name, "생년월일": ""})

# 생일 기준 정렬
valid_births_sorted = sorted(valid_births, key=lambda x: x["생년월일"])
final_output = valid_births_sorted + nan_births

# DataFrame으로 변환 후 출력
final_df = pd.DataFrame(final_output)[["이름", "생년월일"]]
print("📋 조원 생일 정렬 결과:")
display(final_df)


Saving DS_Birthdaydata - 시트1.csv to DS_Birthdaydata - 시트1 (1).csv
📋 조원 생일 정렬 결과:


,이름,생년월일
0,선예영,20030122
1,김하린,20030212
2,이승주,20041005
3,조승연,20050119
4,이서진,20051028
5,황채원,
6,나현주,
7,윤서현,
8,남수연,
9,김민선,


## 코드해설
CSV 업로드 방식->files.upload() 사용하여 Colab에서 직접 업로드 가능하도록 처리!!


1️⃣	교재 코드 (circularDoublyLinkedList.py) 사용	✅	BidirectNode, CircularDoublyLinkedList, Iterator 클래스 모두 교재 형식에 맞게 구현됨

2️⃣	생일 데이터를 리스트 구조에 저장	✅	birthday_list = CircularDoublyLinkedList() 에 append를 통해 저장

3️⃣	같은 조 친구 10명만 필터링	✅	group_members 리스트에 조원 10명 정확히 포함되어 필터링 진행

4️⃣	조원의 이름과 생년월일 출력	✅	final_df = DataFrame[["이름", "생년월일"]] 로 구성되어 출력

5️⃣	출력 결과가 Colab 셀에 나타나야 함	✅	from IPython.display import display + display(final_df) 로 셀에 깔끔하게 출력

🔁 (부가 조건)	생일 누락 시 예외 처리 및 공백 처리	✅	NaN 생일은 "" 처리되어 출력에 오류 없음

🔁 (부가 조건)	생일 빠른 순서로 정렬	✅	sorted(valid_births, key=lambda x: x["생년월일"]) 을 통해 정렬 완료

🔁 (부가 조건)	이서진 중 학번이 44로 끝나는 사람만 추출	✅	"이서진" and 학번.endswith("44") 조건 처리 포함됨

과제3 & 4에서 사용한 파일: [DS_Birthdaydata - 시트1.csv]

#**과제5(교재 8장 우선 순위 큐 연습문제)**

##문제1

In [ ]:
임의의 최대 힙에서 더 얕은 곳에 있는 원소가 더 깊은 곳에 있는 원소보다 더 작은 값을 가질 수 있다.
예를 들어보면, heap[1] = 7, heap[2] = 10, heap[5] = 9인 힙이 있다. heap[1]과 heap[2]는 깊이가 같고, heap[5]는 heap[1]보다 깊이가 깊지만 값은 작다. heap[5]는 heap[2]의 자식 노드로, heap[2] > heap[5]이기 때문에 힙의 조건을 만족한다.

##문제2

In [ ]:
최대 힙의 마지막 원소가 항상 가장 작은 값은 아니다.
heap[0] = 10, heap[1] = 7, heap[2] = 9이고 이 두 노드가 말단 노드인 힙이 있다고 한다. 이 힙은 heap[0] > heap[1], heap[0] > heap[2]가 되어 힙의 조건을 만족하며, heap[n-2] < heap[n-1]을 만족하는 힙이다.

##문제3

In [ ]:
길이가 n인 임의의 리스트를 힙으로 만들 때, 마지막 요소의 인덱스는 n-1이다. 따라서 인덱스가 ((n-1)-1)//2 즉 (n-2)//2부터 0까지의 인덱스를 갖는 요소를 대상으로 스며내리기를 진행한다.
따라서 루트의 자격으로 스며내리기를 하지 않고 그냥 넘어가는 원소 수는 정확하게 n-1-(n-2)//2개이다.

##문제4

In [ ]:
길이가 n인 리스트를 대상으로 하는 스며내리기 알고리즘에서 최악의 경우 Θ(logn)의 시간이 소요된다. 이진 탐색 트리의 깊이에 따라 층을 나누면 층의 개수는 약 (logn + 1)개가 된다. 스며내리기의 대상이 되는 리스트의 인덱스가 0인 요소가 이 리스트의 가장 작은 값이라면 최악의 경우가 되어, 스며내리기를 logn번 진행해야 한다.
최선의 경우에는 Θ(1)의 시간이 소요된다. 스며내리기의 대상이 되는 리스트의 인덱스가 0인 요소가 이 리스트의 가장 큰 값이라면 인덱스가 1, 2인 요소보다 크기 때문에 스며내리기는 진행되지 않고 멈춘다.

##문제5

In [ ]:
이 문제에서 말하는 마지막 원소가 단순히 리스트의 마지막 원소라면 매우 간단히 원소를 삭제할 수 있다.

하지만 여기서 말하는 마지막 원소가 우선순위의 마지막이 되는 원소라면 힙의 마지막 원소를 삭제하는 작업은 그렇게 간단하지는 않다. 연습문제 2번에서 확인한 바와 같이 마지막 원소가 항상 가장 작은 값을 가지지도 않고, 연습문제 1번에서 확인한 바와 같이 깊이가 깊은 원소가 항상 깊이가 얕은 원소보다 작은 값을 가지지도 않는다.
하지만 힙의 특성상 자식 노드는 부모 노드보다 작은 값을 가지기 때문에, 깊이가 2인 모든 서브 트리들의 원소들만 비교하면 가장 마지막 원소를 어렵지 않게 찾을 수 있다. 겨우 깊이가 2인 서브 트리이기 때문에 원소의 삭제 작업도 간단하다.

##문제6

In [ ]:
위쪽부터 시작해서 스며오르기를 반복하여 build_heap() 알고리즘을 구현한다고 해보자. index가 0인 요소에는 이미 맨 위의 요소이므로 스며오르기를 할 수 없다. 따라서 index가 1인 요소부터 index가 n-1(마지막)인 요소까지 진행한다. 처음 스며오르기를 진행할 땐 대상 노드의 깊이가 얕기 때문에 비교와 교체 작업이 많지 않지만, index가 커질수록 대상 노드의 깊이가 깊어져 스며오르기를 logn번 진행해야 할 수도 있다.
n/2 * logn + n/4 * log(n-1) + n/8 * log(n-2) + ... + n/(2**(n-1)) * log2 즉 Θ(nlogn)의 시간이 소요된다. 이 방법은 Θ(n)의 시간이 소요되는 스며내리기를 이용한 build_heap() 알고리즘보다 비효율적이다.

##문제7

In [ ]:
임의의 원소의 값이 증가했다면, 그 원소에 해당하는 노드를 제거하는 delete_max() 작업을 실행한다. 이 작업에 소요되는 시간은 O(logn)이다. 그리고 증가한 원소의 값을 갖는 노드를 힙에 추가한다. 노드를 추가하는 작업 insert()는 O(logn)의 시간이 든다.
이 두 작업을 통해 힙을 O(logn) 시간만에 변화를 반영하여 힙을 수선할 수 있다.

#**과제6 LeetCode 703.Kth Largest Element in Stream**

In [ ]:
public class KthLargest {

    private final PriorityQueue<Integer> q;
    private final int k;

    public KthLargest(int k, int[] nums) {
        this.k = k;
        q = new PriorityQueue<Integer>(k);
        for (int a :nums) {
            add(a);
        }
    }

    public int add(int val) {
        q.offer(val);
        if (q.size() > k) {
            q.poll();
        }
        return q.peek();
    }
}

##풀이해설
우선순위 큐를 이용하여 k사이즈보다 큰 값은 poll해가며 우선순위를 유지한뒤 가장 큰값을 peek 해줍니다.